Carga de librerías

In [1]:
import os
import ingest_flights as src

from google.cloud import storage

Inspección rápida del URL de origen de datos

In [2]:
src.SOURCE

'https://storage.googleapis.com/data-science-on-gcp/edition2/raw'

# Pasos de arquitectura

## Descarga de datos

In [3]:
out_file = src.download(year='2018', month='10', destdir=os.path.join('data','raw'))
out_file

'data\\raw\\201810.zip'

## Descompresión

In [4]:
unziped = src.zip_to_csv(
    filename=out_file
    ,destdir=os.path.join('data','unziped'))

unziped

'data\\unziped\\On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_10.csv.gz'

## Bucket de Cloud Storage

Documentación en creación de [Buckets](https://cloud.google.com/storage/docs/creating-buckets#storage-create-bucket-client_libraries) usando clientes como Python

Documentación sobre las [clases](https://cloud.google.com/storage/docs/storage-classes) de almacenamiento

ID de proyecto:

```sh
gcloud config get-value project
```

In [5]:
PROJECT = 'ds-on-gcp-402120'
client = storage.Client(project=PROJECT)
bucket = client.bucket(bucket_name='ch02')
bucket.storage_class = 'COLDLINE'

if bucket.exists():
    print("Bucket {} with storage class {}".format(
        bucket.name, bucket.storage_class))
    
else:
    bucket = client.create_bucket(bucket, project=PROJECT, location='us-central1')

    print("Created bucket {} in {} with storage class {}".format(
        bucket.name, bucket.location, bucket.storage_class))

Created bucket ch02 in US-CENTRAL1 with storage class COLDLINE


## Ingesta de datos a Cloud Storage

In [12]:
src.upload(
    csvfile=unziped[:-3]
    ,bucketname=bucket.name
    ,blobname='201810.csv')

'gs://ch02/201810.csv'